##### All the Imports required for the script

In [ ]:
import os
import glob
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##### Reading all the files and putting them together as a single dataframe

In [ ]:
path = '/home/mails_info' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
frame.columns

In [ ]:
frame.groupby('label')['next_page_id'].count()

##### Filling in the unprocessed info

In [ ]:
frame_processed_messages = frame[frame['date'].notna()]
joined_frame = frame.merge(frame_processed_messages, on='message_id', suffixes=("_global", "_processed"))
columns_required = ["message_id", "sender_processed", "receiver_processed",
                    "date_processed", "subject_processed", "label_global"]
frame_with_info = joined_frame[columns_required]
rename_columns = ["message_id", "sender", "receiver",
                  "date", "subject", "label"]
frame_with_info.columns = rename_columns
frame_with_info.head(10)

In [ ]:
def extract_date(x):
    """
    Function to Extract the Dates from two different date strings
    So that pd.to_datetime will be able to parse it
    """
    regex = r"\d+ \D{3} \d+ \d{2}:\d{2}:\d{2}|\D{3}, \d+ \D{3} \d+ \d+:\d+:\d+"
    found = re.findall(regex, str(x))
    if found:
        return found[0]
    else:
        return np.nan
    
frame_with_info['clean_date'] = frame_with_info['date'].map(extract_date)

In [ ]:
pd.set_option('display.max_rows', 500)
frame_with_info[frame_with_info['clean_date'].isna() & frame_with_info['date'].notna()]

In [ ]:
frame_with_info['_date'] = pd.to_datetime(frame_with_info['clean_date'])
frame_with_info['only_date'] = frame_with_info['_date'].dt.strftime('%Y-%m-%d')
frame_with_info.head(10)

##### Extracting the sender_email, sender_name and domain from sender information

In [ ]:
frame_with_info['sender_email'] = frame_with_info['sender'].str.extract("([\w\.\=\-]+@[\w\-\.]+)")
frame_with_info['sender_name'] = frame_with_info['sender'].apply(lambda x: re.findall(r"([\s|\w|\W]+ )(<)", str(x))[0][0] \
                                if len(re.findall(r"([\s|\w|\W]+ )(<)", str(x))) else str(x))
frame_with_info['domain'] = frame_with_info['sender'].apply(lambda x: re.findall(r"(@)([\w|\.\-]+)", str(x))[0][1] \
                                if len(re.findall(r"(@)([\w|\.\-]+)", str(x))) else str(x))
frame_with_info.head(10)

##### Let's do a group concat on label

In [ ]:
grouped_info = frame_with_info.groupby(["message_id","sender_email", "sender_name", "domain","subject", "only_date"]).\
apply(lambda x: ",".join(x.label))
grouped_info = grouped_info.to_frame()
grouped_info.reset_index(inplace=True)
grouped_info.head(10)

##### Plotting the pie chart for top 10 domains

In [ ]:
grouped_domain_info = grouped_info.groupby(['domain'])['domain'].count()\
                     .reset_index(name='count')\
                     .sort_values(by='count', ascending=False).head(10)
plt.pie(grouped_domain_info["count"], labels = grouped_domain_info["domain"], radius=2.5)


##### Finding the frequency based on months and year

In [ ]:
grouped_info['_date'] = pd.to_datetime(grouped_info['only_date'], format='%Y-%m-%d')
grouped_info['year_month'] = grouped_info['_date'].dt.strftime("%Y-%m")
grouped_info['year'] = grouped_info['_date'].dt.strftime("%Y")
grouped_info['month'] = grouped_info['_date'].dt.strftime("%m")
grouped_year_month_info = grouped_info.groupby(['year', 'month'])['message_id'].count()\
                     .reset_index(name='count')\
                     .sort_values(by=['year','month']).head(200)
grouped_year_month_info.head(10)

##### Creating the Pivot before plotting

In [ ]:
table = pd.pivot_table(grouped_year_month_info, values='count', index=['month'],
                    columns=['year'], aggfunc=np.sum)
df = pd.DataFrame(table.to_records())
df

In [ ]:
df.columns = [header for header in df.columns]
df.set_index('month', inplace=True)
df.fillna(0)
df.plot.bar(figsize=(17,8), rot=0, width=0.8)